# Binary Classification using Tensorflow

TensorFlow is a versatile framework for machine learning, capable of tackling diverse tasks, including binary classification. Using the Titanic dataset, the objective is to predict whether a passenger survived or not, based on features like:
- **Age**, **Gender**, **Passenger Class (Pclass)**, **Fare**, **Embarked Port**, and more.

### Key Steps in the Process

1. **Data Preprocessing**  
   - Cleaning and handling missing values.  
   - Encoding categorical variables (e.g., gender or embarkation ports).  
2. **Model Building**  
   - Designing a neural network specifically for binary classification.  
   - Using layers like Dense, Activation, and Dropout to create a flexible model.  
3. **Training and Evaluation**  
   - Leveraging TensorFlow's tools to train the model using labelled data.  
   - Evaluating its performance on unseen data to ensure generalisation.
------
We are going to do similar to the Regression Tensorflow workbook, but this time we are going to work at a classification problem

Back to the titanic.csv file we used before

In [39]:
# Import dependencies
import tensorflow as tf # TensorFlow - Main library for building and training neural networks
import pandas as pd # pandas - for data manipulation and handling structured datasets
import numpy as np # NumPy - support for numerical operations and working with arrays
import matplotlib.pyplot as plt # Matplotlib - for visualizing data and model performance
from sklearn.model_selection import train_test_split # utility from scikit-learn to split data into training and test sets

# Print TensorFlow version to confirm installation and compatibility
print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.18.0


### Read and Clean Data

To prepare the dataset for modelling, we'll follow these steps:

1. **Read in the data**: Load the Titanic dataset using a suitable library (e.g., `pandas`).
2. **Handle missing values**:
   - Remove rows where the `Embarked` column is not filled.
   - Fill in missing values in the `Age` column with the median age.
3. **Select relevant features**:
   - Identify and keep only the columns required for building the model.
4. **Encode categorical variables**:
   - Use a `LabelEncoder` to convert the `Sex` column from text (e.g., "male", "female") to numbers.
   - Use a `OneHotEncoder` to encode the `Embarked` column into multiple binary columns (one per embarkation point).


In [40]:
# Read and clean Data ----------
# Load Titanic dataset
df = pd.read_csv("titanic.csv")
# Remove rows with missing 'Embarked' values and fill missing 'Age' with the median
df = df[df["Embarked"].notnull()]
df['Age'] = df['Age'].fillna(df['Age'].median())
# Define target variable 'y' (Survived) and features 'X'
y = df["Survived"]
X = df[["Pclass", "Age", "Sex", "SibSp", "Parch", "Fare", "Embarked"]]

Convert sex to numeric values using label encoder from preprocessing library

In [41]:
from sklearn import preprocessing

# Encode the 'Sex' column using LabelEncoder
le_sex = preprocessing.LabelEncoder()  # Initialize the encoder
le_sex.fit(X['Sex'])                   # Fit the encoder on the 'Sex' column
sex = le_sex.transform(X['Sex'])       # Transform 'Sex' into numerical values (e.g., 0 for male, 1 for female)

# Replace the original 'Sex' column with the encoded values
X = X.drop(['Sex'], axis=1)  # Drop the original 'Sex' column
X['Sex'] = sex               # Add the encoded 'Sex' column back


Convert Embarked to numeric values using OneHotEncoder from preprocessing library. There's probably a better way of doing this but this'll work

In [42]:
from sklearn import preprocessing

# Encode the 'Embarked' column using OneHotEncoder
le_embark = preprocessing.OneHotEncoder(sparse_output=False)  # Initialize encoder with dense output
le_embark.fit(X["Embarked"].values.reshape(-1, 1))            # Fit encoder on the 'Embarked' column
embarked = le_embark.transform(X["Embarked"].values.reshape(-1, 1))  # Transform 'Embarked' into one-hot vectors

# Replace the original 'Embarked' column with one-hot encoded columns
X = X.drop(["Embarked"], axis=1)      # Drop the original 'Embarked' column
X["EmbarkC"] = embarked[:, 0]         # Add column for 'C' (Cherbourg)
X["EmbarkQ"] = embarked[:, 1]         # Add column for 'Q' (Queenstown)
X["EmbarkS"] = embarked[:, 2]         # Add column for 'S' (Southampton)
X  # Display the updated DataFrame

,Pclass,Age,SibSp,Parch,Fare,Sex,EmbarkC,EmbarkQ,EmbarkS
0,3,22.0,1,0,7.2500,1,0.0,0.0,1.0
1,1,38.0,1,0,71.2833,0,1.0,0.0,0.0
2,3,26.0,0,0,7.9250,0,0.0,0.0,1.0
3,1,35.0,1,0,53.1000,0,0.0,0.0,1.0
4,3,35.0,0,0,8.0500,1,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
886,2,27.0,0,0,13.0000,1,0.0,0.0,1.0
887,1,19.0,0,0,30.0000,0,0.0,0.0,1.0
888,3,28.0,1,2,23.4500,0,0.0,0.0,1.0
889,1,26.0,0,0,30.0000,1,1.0,0.0,0.0


### Machine Learning Process

Now we can proceed with the Machine Learning workflow. The steps include:

1. **Train-Test Split**: Divide the dataset into training and testing sets to evaluate model performance on unseen data.
2. **Normalise the Data**: Scale the features to ensure consistent ranges, improving model convergence during training.
3. **Build Base Models**: Define initial neural network architectures for binary classification.
4. **Compile**: Configure the model with an optimiser, loss function, and evaluation metrics.
5. **Fit**: Train the model on the training data, adjusting weights based on the loss function.
6. **Evaluate**: Test the model on the test set to assess its performance.

These steps follow the typical train-test split approach to ensure fair evaluation and generalisation of the model.


In [43]:
# train/test split (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1138, train_size=0.8)

Normalisation like was done on the Regression workbook

In [44]:
###Error### >from tensorflow.keras.layers.experimental import preprocessing<
# The tensorflow.keras.layers.experimental.preprocessing module has been deprecated in recent TensorFlow versions. 

# The Normalization layer is now part of tensorflow.keras.layers. Here's the corrected code:
from tensorflow.keras.layers import Normalization
# Initialize the Normalization layer
normalizer = Normalization()
# Adapt the normalizer to the training data (compute mean and variance)
normalizer.adapt(X_train.values)
# Apply normalization to the training data and print the result
print('Normalized:', normalizer(X_train.values).numpy())

Normalized: [[-1.5950363   0.1471719  -0.46338156 ... -0.4819084  -0.3198837
   0.6256112 ]
 [ 0.80765593 -1.572492   -0.46338156 ... -0.4819084  -0.3198837
   0.6256112 ]
 [ 0.80765593 -0.08732773 -0.46338156 ...  2.0750833  -0.3198837
  -1.5984368 ]
 ...
 [-1.5950363  -0.08732773 -0.46338156 ... -0.4819084  -0.3198837
   0.6256112 ]
 [ 0.80765593 -0.7126601  -0.46338156 ... -0.4819084  -0.3198837
   0.6256112 ]
 [-0.39369014 -2.1978242   1.3031831  ... -0.4819084  -0.3198837
   0.6256112 ]]


## Model Number 1 - Logistic Regression
We will begin with a simple Logistic Regression model as a baseline.
### Key Characteristics:
- Logistic Regression is a linear model used for binary classification.
- The output layer consists of **one unit** (neuron) with a sigmoid activation function to produce probabilities.
Mathematically, the model represents:
$$\log\left(\frac{p}{1-p}\right) = w^Tx + b$$
This formulation calculates the log-odds of the probability \(p\), where \(w\) represents the weights, \(x\) is the input data, and \(b\) is the bias term.
### TensorFlow Implementation:
- The model architecture will include a normalisation layer to scale the input data before applying the logistic regression layer.
- This approach is conceptually similar to using `sklearn`'s `LogisticRegression()`.

Including the normaliser as a layer ensures data is processed consistently during both training and inference.


In [45]:
# Define a Sequential model for logistic regression
model = tf.keras.Sequential([
    normalizer,  # Normalization layer to preprocess input data
    # Logistic regression layer
    tf.keras.layers.Dense(1)
])

# Display the model summary
model.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization_3 (Normalization) │ (711, 9)               │            19 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19 (80.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 19 (80.00 B)

### Understanding Model Output and Loss Function

1. **Trainable Parameters**:  
   - The model has **10 trainable parameters** because we have:
     - **9 features** (one weight for each feature).
     - **1 bias** term (\(b\)). (probability of survival if all weights are 0)

2. **Loss Function**:  
   - We use `BinaryCrossentropy` from TensorFlow because this is a binary classification problem.  
   - Documentation: [BinaryCrossentropy](https://www.tensorflow.org/api_docs/python/tf/keras/losses/BinaryCrossentropy).

3. **Using `from_logits`**:  
   - The `from_logits` parameter determines whether the output of the model is **logits** (unscaled values) or **probabilities** (scaled between 0 and 1).  
   - If no activation function is applied to the output layer (e.g., no `sigmoid`), the output is **logits**.  
     - In this case, set `from_logits=True`. This is **recommended** by TensorFlow for better numerical stability.  
   - If you apply a `sigmoid` activation in the output layer (e.g., `Dense(1, activation='sigmoid')`), the output will be probabilities.  
     - Here, set `from_logits=False`.

4. **Conclusion**:  
   - In the current implementation (`Dense(1)` without an activation), the model outputs logits, so you should use `from_logits=True` in the loss function.
   - If you decide to use `Dense(1, activation='sigmoid')`, update the loss function with `from_logits=False` (though TensorFlow recommends avoiding this setup for stability reasons).

**Key Tip**: Always align the `from_logits` parameter in the loss function with the activation used (or not used) in the model's output layer.


In [46]:
# Compile the model with optimizer, loss function, and evaluation metrics
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),  # Adam optimizer with learning rate 0.1
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),  # Binary cross-entropy loss with logits
    metrics=['accuracy']  # Track accuracy during training -- it must be in []
)

# Train the model on the training data for 100 epochs
model.fit(X_train, y_train, epochs=100)


Epoch 1/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6043 - loss: 0.7130  
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7964 - loss: 0.4653 
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7856 - loss: 0.4652 
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8177 - loss: 0.4606 
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8106 - loss: 0.4263 
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8064 - loss: 0.4254 
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8198 - loss: 0.4158 
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8026 - loss: 0.4660 
Epoch 9/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8127 - loss: 0.4585 
Epoch 10/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8195 - loss: 0.4194 
Epoch 11/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7814 - loss: 0.4754 
Epoch 12/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

### Observations on Training Output

1. **Loss Decreasing**:  
   - The loss decreases consistently across epochs, indicating the model is learning from the training data. However, since no validation set is used, we cannot assess if the model is overfitting.

2. **Number of Epochs**:  
   - The model was trained for 100 epochs, which might be excessive or insufficient depending on the dataset. Using a validation set and monitoring metrics like validation loss can help determine an optimal number of epochs.

3. **Caution on `.fit`**:  
   - If you call `.fit` on the same model again, training will **continue from where it left off** (e.g., epoch 101 onwards), as the optimizer retains its state.  
   - To start training from scratch, you need to reinitialize the model or reset its weights.

4. **Improvements for Future Runs**:  
   - Use a **validation set** (e.g., via `validation_split` or a separate test set) to monitor performance and detect overfitting.
   - Implement **early stopping** to halt training once the validation loss stops improving, saving computation and avoiding overfitting.

5. **Accuracy Plateau**:  
   - Accuracy fluctuates slightly around a certain range (e.g., 0.81-0.83), which might indicate the model has reached its capacity with the given architecture and features.
   - Consider exploring hyperparameter tuning, additional features, or more complex models if further improvement is needed.


In [59]:
# Evaluate the logistic regression model on the test set
logRegEval = model.evaluate(X_test, y_test)

# Print the test loss and accuracy, rounded for readability
print("\nLogistic Regression Loss: ", round(logRegEval[0], 4), # logRegEval[0]: The loss from model.evaluate() rounded to 4 decimals.
      "Accuracy: ", round(logRegEval[1], 4)) # logRegEval[1]: The accuracy from model.evaluate() rounded to 4 decimals.

logRegEval # raw output loss first, accuracy second


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7708 - loss: 0.5493 

Logistic Regression Loss:  0.516 Accuracy:  0.7865


[0.5160284638404846, 0.7865168452262878]

### Output Analysis

**Accuracy**: 78.65%  
- Slightly lower than training (~83%), indicating mild overfitting.

**Loss**: 0.516  
- Moderate, showing predictions are reasonably close to true labels.

### Suggestions
- Use a validation set to monitor overfitting.
- Apply regularisation or tune hyperparameters.


# ---------------------------- Folyt!!! -----------------------------------
## Model Number 2 - Neural Network

Now we are going to build a Neural Network

4 layers, 100 units in each hidden layer with relu as the activation function (you can try different structures if you want but for now I'm just demonstrating how Tensorflow works as well as some other things we can try)

In [48]:
model_tf = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(100,activation='relu'),
    tf.keras.layers.Dense(100,activation='relu'),
    tf.keras.layers.Dense(100,activation='relu'),
    tf.keras.layers.Dense(1)
])

In [49]:
model_tf.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization_3 (Normalization) │ (None, 9)              │            19 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19 (80.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 19 (80.00 B)

21301 trainable params, a lot more than the previous 10! Let's look at each layer

Input has 9 features Hidden Layer 1 has 100 units. Therefore the weight matrix is going to have 100 rows and 9 columns to match
$$ Wx + b $$

That gives us 900 paramaters for the weights. Then we have 100 biases - giving the total of 1000 parameters to go from input layer to hidden layer 1

Now from hidden layer 1 to hidden layer 2. 100 features, 100 units, gives us a 100x100 matrix so 10000 parameters, add in the 100 biases and we get the 10100

Let's compile and fit

In [50]:
model_tf.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.01),
    metrics='accuracy',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True)
)

model_tf.fit(X_train, y_train, epochs=100)

ValueError: Expected `metrics` argument to be a list, tuple, or dict. Received instead: metrics=accuracy of type <class 'str'>

Again Loss keeps decreasing, haven't use a validation set though. Could've done a different number of epochs

In [ ]:
print("Logistic Regression Loss: ", round(logRegEval[0],4), "Accuracy: ", round(logRegEval[1],4))

In [ ]:
annEval = model_tf.evaluate(X_test, y_test)
print("ANN Loss: ", round(annEval[0],4), "Accuracy: ", round(annEval[1],4))

These are potentially overfit, we should have done some validation to check over things

While the accuracy for the test data is better, the loss is actually worse!

## Using Validation
### Logistic Regression with Validation

Let's add in validation. kFold Cross Validation would be nice but that's more difficult to do with tensorflow and requires writing our own functions so let's just take the last 20% as a validation set. Since we used train_test_split already, the data is already shuffled so it should be ok

In [ ]:
model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(1)
])
model.summary()

In [ ]:
model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    metrics='accuracy',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True)
)

history = model.fit(X_train, y_train, epochs=100, validation_split=0.2)

Same function that was in the previous notebook

In [ ]:
def plot_loss(history, which='loss'):
    plt.plot(history.history[which], label='train')
    try:
        plt.plot(history.history['val_'+which], label='validation')
    except:
        None
    plt.xlabel('Epoch')
    plt.ylabel(which)
    plt.legend()
    plt.grid(True)

In [ ]:
plot_loss(history)

In [ ]:
logRegValidationEval = model.evaluate(X_test, y_test)
print("Logistic Regression with Validation Loss: ", round(logRegValidationEval[0],4), "Accuracy: ", round(logRegValidationEval[1],4))

Seems about the same performance on the test set. Your numbers will vary due to SGD

Now ANN

### ANN with Validation

In [ ]:
model_tf = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(100,activation='relu'),
    tf.keras.layers.Dense(100,activation='relu'),
    tf.keras.layers.Dense(100,activation='relu'),
    tf.keras.layers.Dense(1)
])

model_tf.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.01),
    metrics='accuracy',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True)
)

history = model_tf.fit(X_train, y_train, epochs=100, validation_split=0.2)

Let's plot the loss graph

In [ ]:
plot_loss(history)

Validation loss is a bit all over the place. This suggests overfitting to me

Neural Networks are not always better....even though the training loss and accuracy was better with the NN

In [ ]:
plot_loss(history, 'accuracy')

In [ ]:
print("Logistic Regression with Validation Loss: ", round(logRegValidationEval[0],4), "Accuracy: ", round(logRegValidationEval[1],4))

In [ ]:
annValidationEval = model_tf.evaluate(X_test, y_test)
print("ANN with Validation Loss: ", round(annValidationEval[0],4), "Accuracy: ", round(annValidationEval[1],4))

The NN did do better with the test set in terms of accuracy but loss quite a bit worse

## ANN with Regularisation

Ok, let's try smoothing some of that out with adding l2 regularisation to the layers

In [ ]:
#Defining model with L2 regularisation
model_tf = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(100,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01), bias_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(100,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01), bias_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(100,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01), bias_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(1)
])

model_tf.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.01),
    metrics='accuracy',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True)
)

history = model_tf.fit(X_train, y_train, epochs=100, validation_split=0.2)

val_loss: 0.4876 - val_accuracy: 0.8462 , so better than the last NN we build

In [ ]:
plot_loss(history)

That graph looks a lot better with some regularisation

In [ ]:
model_tf.evaluate(X_test, y_test)
annValidationRegularisationEval = model_tf.evaluate(X_test, y_test)
print("ANN with Regularisation & Validation Loss: ", round(annValidationRegularisationEval[0],4), "Accuracy: ", round(annValidationRegularisationEval[1],4))

In [ ]:
print("Logistic Regression Loss: ", round(logRegEval[0],4), "Accuracy: ", round(logRegEval[1],4))
print("ANN Loss: ", round(annEval[0],4), "Accuracy: ", round(annEval[1],4))
print("Logistic Regression with Validation Loss: ", round(logRegValidationEval[0],4), "Accuracy: ", round(logRegValidationEval[1],4))
print("ANN with Validation Loss: ", round(annValidationEval[0],4), "Accuracy: ", round(annValidationEval[1],4))
print("ANN with Regularisation & Validation Loss: ", round(annValidationRegularisationEval[0],4), "Accuracy: ", round(annValidationRegularisationEval[1],4))

Interesting to see that ANN doesn't do the best in terms of loss but does well in accuracy performace - shows that optimising for loss and accuracy are not the same thing. Also shows that a simpler model can sometimes be better choice